In [1]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 5.0 MB/s eta 0:00:00


### Here we used NN with 1 hidden layer and our Future Forecast method

In [2]:
import pandas as pd
import numpy as np
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense, LSTM
import tensorflow_addons as tfa
import os
import tensorflow as tf
from tensorflow.keras import layers
%matplotlib inline

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


### Read Dataset

In [3]:
data = pd.read_csv('/content/train.csv')
data

,id,date,year_week,product_number,reporterhq_id,prod_category,specs,display_size,segment,sales_units,inventory_units
0,202119-6909,2021-05-15,202119,6909,93,Arale,21274,13.3,Premium,2.0,35.0
1,202120-6909,2021-05-22,202120,6909,93,Arale,21274,13.3,Premium,0.0,70.0
2,202121-6909,2021-05-29,202121,6909,93,Arale,21274,13.3,Premium,3.0,137.0
3,202122-6909,2021-06-05,202122,6909,93,Arale,21274,13.3,Premium,0.0,274.0
4,202123-6909,2021-06-12,202123,6909,93,Arale,21274,13.3,Premium,0.0,333.0
...,...,...,...,...,...,...,...,...,...,...,...
15449,202317-247737,2023-04-29,202317,247737,15,Arale,10637,13.3,Premium,0.0,62.0
15450,202317-247737,2023-04-29,202317,247737,24,Arale,10637,13.3,Premium,0.0,3.0
15451,202318-247737,2023-05-06,202318,247737,3,Arale,10637,13.3,Premium,22.0,0.0
15452,202318-247737,2023-05-06,202318,247737,15,Arale,10637,13.3,Premium,68.0,78.0


# Data Processing

Cast column types

In [5]:
# Define the column type mapping
cate_columns = ['id', 'year_week', 'product_number', 'reporterhq_id',
       'prod_category', 'specs', 'display_size', 'segment']
data[cate_columns] = data[cate_columns].astype('str')
data['date'] = pd.to_datetime(data['date'])

In [9]:
dfs = []
config = {
    "236880":"202201",
    "148050":"202243",
    "12831":"202221",
    "48363":"202233",
    "157920":"202228",
    "190491":"202227",
    "46389":"202204",
    "116466":"202217",
    "234906":"202204"
}
product_ids = data.product_number.unique()
for product_id in product_ids:
  if product_id in list(config.keys()):
    abc = data[(data['product_number'] == product_id) & (data['year_week'] >= config[product_id]) & (data['year_week'] <= '202318')]
    dfs.append(abc)
  dfs.append(data[data['product_number'] == product_id])
df_col_merged = pd.concat(dfs, axis=0)
df_col_merged.head(2)

,id,date,year_week,product_number,reporterhq_id,prod_category,specs,display_size,segment,sales_units,inventory_units
0,202119-6909,2021-05-15,202119,6909,93,Arale,21274,13.3,Premium,2.0,35.0
1,202120-6909,2021-05-22,202120,6909,93,Arale,21274,13.3,Premium,0.0,70.0


In [11]:
def generate_year_week_list(start_year, start_week, end_year=2023, end_week=18):
    start_date = datetime.datetime.strptime(f'{start_year}-W{start_week}-1', '%Y-W%W-%w')
    end_date = datetime.datetime.strptime(f'{end_year}-W{end_week}-1', '%Y-W%W-%w')
    current_date = start_date
    year_week_list = []
    
    while current_date <= end_date:
        year = current_date.isocalendar()[0]
        week = current_date.isocalendar()[1]
        year_week_list.append((year, f'{week:02}'))  # Format week number with leading zeros
        current_date += datetime.timedelta(weeks=1)

    df = pd.DataFrame(year_week_list, columns=['Year', 'Week'])
  
    return df

In [14]:
import datetime
df_col_mergedd = df_col_merged.groupby(['date', 'year_week', 'product_number', 'prod_category', 'display_size', 'segment']).agg({'sales_units': 'sum', 'inventory_units':'sum'}).reset_index()

dfs = []
for product_id in df_col_mergedd.product_number.unique():
    da = df_col_mergedd[df_col_mergedd['product_number']==product_id]

    da = da.sort_values("year_week", ascending=True)
    year_week = da.head(1)['year_week'].values[0]

    start_year = year_week[:4]
    start_week = year_week[-2:]

    ab = generate_year_week_list(int(start_year),int(start_week))
    ab["year_week"] = ab["Year"].astype(str) + ab['Week'].astype(str)
    
    abc = ab.merge(da, on='year_week', how='left')
    dfs.append(abc)

df_col = pd.concat(dfs, axis=0)
df_col.fillna(method='ffill', inplace = True)

,Year,Week,year_week,date,product_number,prod_category,display_size,segment,sales_units,inventory_units
0,2019,16,201916,2019-04-20,233919,Clover,15.6,Core,3.0,43.0
1,2019,17,201917,2019-04-27,233919,Clover,15.6,Core,122.0,184.0
2,2019,18,201918,2019-05-04,233919,Clover,15.6,Core,61.0,219.0
3,2019,19,201919,2019-05-11,233919,Clover,15.6,Core,33.0,133.0
4,2019,20,201920,2019-05-18,233919,Clover,15.6,Core,142.0,310.0
...,...,...,...,...,...,...,...,...,...,...
27,2023,14,202314,2023-04-08,107583,Clover,15.6,Core,16.0,504.0
28,2023,15,202315,2023-04-15,107583,Clover,15.6,Core,77.0,245.0
29,2023,16,202316,2023-04-22,107583,Clover,15.6,Core,136.0,443.0
30,2023,17,202317,2023-04-29,107583,Clover,15.6,Core,115.0,297.0


In [15]:
def fix_date(data):
  sample = data.sort_values('year_week',ascending=True).reset_index(drop=True)
  for i in range(len(sample)-1):
    sample['date'][i+1] = sample['date'][i] + datetime.timedelta(weeks = 1)
  return sample

In [17]:
df_col['date'] = pd.to_datetime(df_col['date'])
dfs = []
for product_id in df_col.product_number.unique():
    da = df_col[df_col['product_number']==product_id]
    ab = fix_date(da)
    dfs.append(ab)
df_col = pd.concat(dfs, axis=0)
df_col.head(2)

<ipython-input-15-1939b2ea315a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['date'][i+1] = sample['date'][i] + datetime.timedelta(weeks = 1)


,Year,Week,year_week,date,product_number,prod_category,display_size,segment,sales_units,inventory_units
0,2019,16,201916,2019-04-20,233919,Clover,15.6,Core,3.0,43.0
1,2019,17,201917,2019-04-27,233919,Clover,15.6,Core,122.0,184.0


In [18]:
df_col.to_csv('final_data.csv',index=False)

### Future Forecast method

In [26]:
inventory_data = pd.read_csv('/content/final_data.csv', parse_dates=["date"], index_col=["date"]).sort_index()
cate_columns = ['year_week', 'product_number', 'Year', 'Week',
       'prod_category', 'display_size', 'segment']
inventory_data[cate_columns] = inventory_data[cate_columns].astype('str') 

In [27]:
# Generate new columns and get their mean and std values

WINDOW_SIZE=4
list_data = []
for product_id in inventory_data.product_number.unique():
    da = df_col[df_col['product_number']==product_id]

    for i in range(WINDOW_SIZE): # Shift values for each step in WINDOW_SIZE
        da[f"inventory_units+{i+1}"] = da["inventory_units"].shift(periods=i+1)
    da['Mean'] = da[['inventory_units+1', 'inventory_units+2', 'inventory_units+3', 'inventory_units+4']].mean(axis=1)
    da['std'] = da[['inventory_units+1', 'inventory_units+2', 'inventory_units+3', 'inventory_units+4']].std(axis=1)
    list_data.append(da)

clean_data = pd.concat(list_data, axis=0)
clean_data.head(2)

<ipython-input-27-80124b0c5bd6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  da[f"inventory_units+{i+1}"] = da["inventory_units"].shift(periods=i+1)
<ipython-input-27-80124b0c5bd6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  da[f"inventory_units+{i+1}"] = da["inventory_units"].shift(periods=i+1)
<ipython-input-27-80124b0c5bd6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,Year,Week,year_week,date,product_number,prod_category,display_size,segment,sales_units,inventory_units,inventory_units+1,inventory_units+2,inventory_units+3,inventory_units+4,Mean,std
0,2019,16,201916,2019-04-20,233919,Clover,15.6,Core,3.0,43.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,17,201917,2019-04-27,233919,Clover,15.6,Core,122.0,184.0,43.0,NaN,NaN,NaN,43.0,NaN


In [29]:
# Prepare data for training
def prepare_df(df):
    df = df[['product_number', 'prod_category','display_size', 'segment', 'sales_units', "inventory_units+1", "inventory_units+2", "inventory_units+3", "inventory_units+4", "Mean", "std"]]
    df = pd.get_dummies(df, columns=['product_number', 'prod_category','display_size', 'segment'])
    return df
X_all = clean_data.drop(["inventory_units"], axis=1).dropna()
X_train = prepare_df(X_all)
y_train = clean_data.dropna()["inventory_units"]

In [30]:
# Find bes parameters for training

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestRegressor(random_state = 42)
n_estimators = [100, 200, 400, 600] # number of trees in the random forest
max_depth = [5, 10, 15, 20] # maximum number of levels allowed in each decision tree

random_grid = {'n_estimators': n_estimators,

'max_depth': max_depth,
}

rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train,y_train)

In [ ]:
model_1 = rf_random.best_estimator_
# Set random seed for as reproducible results as possible
tf.random.set_seed(42)

# Construct model
model_1 = tf.keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(1)      
], name="model_1_dense") # give the model a name so we can save it

# Compile model
model_1.compile(
                optimizer=tf.keras.optimizers.Adam(lr=0.001),
                loss='mse',
                metrics=[tf.keras.metrics.RootMeanSquaredError()]) 

model_1.fit(x=X_train, # train windows of 4 timesteps of maize prices
            y=y_train, # horizon value of 1
            epochs=100,
            verbose=1,
            batch_size=32,) # create ModelCheckpoint callback to save best model

Epoch 1/100
240/240 [==============================] - 2s 3ms/step - loss: 10728.5215 - root_mean_squared_error: 103.5786
Epoch 2/100
240/240 [==============================] - 1s 3ms/step - loss: 5056.6230 - root_mean_squared_error: 71.1099
Epoch 3/100
240/240 [==============================] - 1s 3ms/step - loss: 4918.0215 - root_mean_squared_error: 70.1286
Epoch 4/100
240/240 [==============================] - 1s 3ms/step - loss: 4776.1162 - root_mean_squared_error: 69.1095
Epoch 5/100
240/240 [==============================] - 1s 3ms/step - loss: 4733.2075 - root_mean_squared_error: 68.7983
Epoch 6/100
240/240 [==============================] - 1s 3ms/step - loss: 4716.4062 - root_mean_squared_error: 68.6761
Epoch 7/100
240/240 [==============================] - 1s 3ms/step - loss: 4717.9966 - root_mean_squared_error: 68.6877
Epoch 8/100
240/240 [==============================] - 1s 4ms/step - loss: 4700.4180 - root_mean_squared_error: 68.5596
Epoch 9/100
240/240 [=================

In [ ]:
model_1.evaluate(X_train, y_train)

240/240 [==============================] - 1s 3ms/step - loss: 4192.0083 - root_mean_squared_error: 64.7457


[4192.00830078125, 64.74571990966797]

### Inference

In [ ]:
# we need to prepare the test data 
# and fill empty values before we can submit it
test_df = pd.read_csv('test.csv')
test_df[['year_week', 'product_number']] = test_df.id.str.split('-', expand = True)
test_df['product_number'] = test_df.product_number
test_df

,id,year_week,product_number
0,202319-105609,202319,105609
1,202319-107583,202319,107583
2,202319-10857,202319,10857
3,202319-109557,202319,109557
4,202319-112518,202319,112518
...,...,...,...
1295,202331-92778,202331,92778
1296,202331-93765,202331,93765
1297,202331-94752,202331,94752
1298,202331-9870,202331,9870


In [ ]:
# each product has specific information, we are just creating a reference table
product_mapping = X_all[['product_number', 'prod_category','display_size', 'segment']].drop_duplicates()
test_df_complete = test_df.merge(product_mapping, on='product_number', how = 'left')

In [ ]:
# Use average mean, std from predicted + last 3 predicted values 

results = {}
for product_id in test_df_complete.product_number.unique():
    feature_values = []
    future_forecast = []
    da = test_df_complete[test_df_complete['product_number']==product_id]
    last_window = list(clean_data[clean_data['product_number']==product_id].inventory_units.values[-WINDOW_SIZE:])
    last_sales = list(clean_data[clean_data['product_number']==product_id].sales_units.values[-WINDOW_SIZE:])
    for i in range(13):
        features = da[['product_number', 'prod_category','display_size', 'segment']].head(1)
        features["inventory_units+1"] = last_window[3]
        features["inventory_units+2"] = last_window[2]
        features["inventory_units+3"] = last_window[1]
        features["inventory_units+4"] = last_window[0]
        features['Mean'] = features[['inventory_units+1', 'inventory_units+2', 'inventory_units+3', 'inventory_units+4']].mean(axis=1)
        features['std'] = features[['inventory_units+1', 'inventory_units+2', 'inventory_units+3', 'inventory_units+4']].std(axis=1)
        mean = np.mean(last_sales)
        features['sales_units'] = [mean]
        last_sales = (last_sales + [mean])[1:]
        prepare_feature = prepare_df(features)
        # Align columns of test set with training set
        missing_cols = set(X_train.columns) - set(prepare_feature.columns)
        for col in missing_cols:
            prepare_feature[col] = 0

        prepare_feature = prepare_feature[X_train.columns]
        pred_value = model_1.predict(prepare_feature)
        # Create an empty list for future forcast
        last_window = (list(pred_value[0]) + last_window)[:-1]
        future_forecast.append(pred_value[0][0])

        # Create a empty list for future feature
        feature_values.append(last_window)
    
    results[product_id] = future_forecast

In [ ]:
results

{'105609': [87.08889,
  109.37649,
  53.49177,
  68.80196,
  88.677025,
  111.033295,
  67.37909,
  68.19325,
  87.2289,
  113.59592,
  79.00581,
  68.86702,
  84.23772],
 '107583': [231.94926,
  260.73267,
  368.77362,
  242.2068,
  210.23448,
  223.19405,
  324.69812,
  239.54391,
  193.24358,
  190.78719,
  285.36847,
  238.4991,
  180.5018],
 '10857': [207.55919,
  147.75772,
  148.87346,
  228.18762,
  197.39201,
  132.57375,
  133.2792,
  207.50633,
  187.84323,
  118.991615,
  118.17865,
  188.0495,
  178.6353],
 '109557': [83.19767,
  80.664116,
  107.11786,
  117.83331,
  81.00839,
  72.57898,
  90.893906,
  110.440285,
  81.31434,
  68.014786,
  78.351906,
  102.11217,
  82.80501],
 '112518': [178.57259,
  168.34315,
  185.9356,
  200.50267,
  174.29713,
  160.76341,
  171.36238,
  189.44688,
  171.34349,
  156.25192,
  160.42368,
  179.00172,
  168.65898],
 '114492': [188.81572,
  152.81006,
  181.92235,
  247.92728,
  174.26651,
  134.77684,
  150.72203,
  216.9893,
  164.5

In [ ]:
import json
with open("result_final.json", "w") as fp:
    json.dump(results,fp) 